Uzitecne python funkce

In [9]:
my_string = "Hello cruel world"
my_list = my_string.split()
print 'Delka stringu je', len(my_string)
print 'String obsahuje slova', my_string.split()
print 'Lowercase ', my_string.lower()
print 'Delka pole je', len(my_list)
print 'Prvni prvek v poli je', my_list[0]
print 'Posledni prvek v poli je', my_list[-1]

Delka stringu je 17
String obsahuje slova ['Hello', 'cruel', 'world']
Lowercase  hello cruel world
Delka pole je 3
Prvni prvek v poli je Hello
Posledni prvek v poli je world


# Spustit interaktivni shell Spark-pythonu

`pyspark --master yarn --num-executors 4`

### word count nebo Hello hadoop

In [ ]:
def split_string(verse):    
    return verse.split(' ')

lines = sc.textFile("/user/pascepet/bible.txt")
words = lines.flatMap(split_string)
pairs = words.map(lambda word: (word, 1))
counts = pairs.reduceByKey(lambda a, b: a + b)

# samostatny ukol c.1

### Ukol 

Spocitat pocet unikatnich slov v kazdem verse a najit vers s nejvetsim poctem slov

### Data

`hdfs:///user/pascepet/bible.txt`

### Ocekavany vystup

| verse_id | pocet_slov |
|:---------|:-----------|
|          |            |

### Bonus odfiltrovat stop-slova pomoci [Spark broadcast](https://spark.apache.org/docs/latest/rdd-programming-guide.html#broadcast-variables)

In [ ]:
# nacist stop slova do mnoziny muzete pomoci prikazu
with open('/tmp/pascepet/stopwords.txt') as stopwords_file:
    stopwords = set([x.replace('\n', '') for x in stopwords_file.readlines()])

# samostatny ukol c.2

### Ukol 
Vypsat top 10 zakazniku z nejvyssi utratou/nejvetsim zustatkem za dane obdobi

### Data 

`hdfs:///user/pascepet/tranzakce/`

### Vstupni data

`csv soubory, oddelovac ','`

| id zakaznika | id protiuctu | castka | je odchozi |
|:-------------|:-------------|:-------|:-----------|
| 1202         | 2456         | 564    | True       |


### Ocekavany vystup

| id zakaznika | aktualni zustatek | utrata | 
|:-------------|:------------------|:-------|
| 1202         | -200              | 450    |



# samostatny ukol c.3

### Ukol
Pro kazdeho zakaznika spocitat podil odchozich volani a pomer poctu kontakty ku poctu volani

### Data

`hdfs:///user/pascepet/spark_sql/cdr_sample/`

### Vstup

| typ rekordu | timestamp | duration | frommsisdn_prefix | frommsisdn | tomsisdn_prefix | tomsisdn |
|:------------|:----------|:---------|:------------------|:-----------|:----------------|:---------|

### Vystup

| frommsisdn | moc_% | number_ratio |
|:-----------|:------|:-------------|


In [ ]:
# data ulozena v parquet formatu, pro nacitani parquet souboru pouzijte 

rdd = sqlContext.read.parquet('hdfs://spark_sql/cdr_sample/').rdd